<p style="font-family: Arial; font-size:2.5em;color:purple; font-style:bold"><br/>
河北民族师范学院网络爬虫
</p>

----------------------------

<p style="font-family: Arial; font-size:1.5em;color:purple; font-style:bold">
冰山
</p>

- 最终看到的结果（程序）只是浮出海面上的部分
- 开发探索的过程是海面之下的部分
- 今天我们将看到冰山的全貌

<p style="font-family: Arial; font-size:1.5em;color:purple; font-style:bold">
鸟瞰整个森林
</p>

- 互联网（Internet）
  - TCP/IP协议
- 万维网（World Wide Web）
  - 超文本传输协议（HTTP）
  - 超文本标记语言（HTML）
  - 统一资源定位符（URL）

<p style="font-family: Arial; font-size:1.5em;color:purple; font-style:bold">
浏览器开发者工具
</p>

- Chrome
- Firefox

<p style="font-family: Arial; font-size:1.5em;color:purple; font-style:bold">
访问一个网页时发生了什么？
</p>

[河北民族师范学院校内新闻](http://www.hbun.net/smartcore/web/main/xnxw/index.htm)

<p style="font-family: Arial; font-size:1.5em;color:purple; font-style:bold">
Python和网络爬虫
</p>

- [Requests](http://python-requests.org)
  - 使用HTTP协议发送网络请求，接收响应


- [BeautifulSoup](http://www.crummy.com/software/BeautifulSoup/bs4/)
  - 从HTML文档中抽取数据

<p style="font-family: Arial; font-size:1.5em;color:purple; font-style:bold">
导入**Requests**和**BeautifulSoup**
</p>

In [ ]:
import requests
import bs4

<p style="font-family: Arial; font-size:1.5em;color:purple; font-style:bold">
获取校内新闻列表页
</p>

In [ ]:
response = requests.get('http://www.hbun.net/smartcore/web/main/xnxw/index.htm')

<p style="font-family: Arial; font-size:1.5em;color:purple; font-style:bold">
查看响应状态
</p>

HTTP状态码

In [ ]:
response.status_code

<p style="font-family: Arial; font-size:1.5em;color:purple; font-style:bold">
设置响应内容文本的字符编码
</p>

In [ ]:
response.encoding = 'utf-8'

<p style="font-family: Arial; font-size:1.5em;color:purple; font-style:bold">
查看响应的内容
</p>

- 一共有多少个字符？
- 打印前100个字符

In [ ]:
len(response.text)

In [ ]:
print(response.text[0:100])

<p style="font-family: Arial; font-size:1.5em;color:purple; font-style:bold">
HTML文档是一棵由标签构成的树
</p>

- 通过浏览器的开发者工具查看
- 使用 **Beautiful Soup** 从中提取数据

<p style="font-family: Arial; font-size:1.5em;color:purple; font-style:bold">
创建BeautifulSoup对象
</p>

用来表示整个HTML文档

In [ ]:
soup = bs4.BeautifulSoup(response.text, 'lxml')

<p style="font-family: Arial; font-size:1.5em;color:purple; font-style:bold">
学校要闻页面的文档结构
</p>

- html
  - body
    - div
      - div
        - div class="sub_rbox"
          - ul
            - li
              - a

<p style="font-family: Arial; font-size:1.5em;color:purple; font-style:bold">
提取新闻标题列表标签
</p>

In [ ]:
ul_tag = soup.find("div", class_="sub_rbox").find("ul")
type(ul_tag)

<p style="font-family: Arial; font-size:1.5em;color:purple; font-style:bold">
提取文章的超链接的列表
</p>

In [ ]:
a_list = ul_tag.find_all('a')

<p style="font-family: Arial; font-size:1.5em;color:purple; font-style:bold">
查看第一篇文章的相对URL
</p>

In [ ]:
a_list[0]['href']

<p style="font-family: Arial; font-size:1.5em;color:purple; font-style:bold">
组装第一篇文章的完整URL
</p>

In [ ]:
from urllib.parse import urljoin
first_url = urljoin(response.url, a_list[0]['href'])
first_url

<p style="font-family: Arial; font-size:1.5em;color:purple; font-style:bold">
发送请求，获取第一篇文章
</p>

In [ ]:
first_response = requests.get(first_url)
first_response.status_code

In [ ]:
first_response.encoding = 'utf-8'
first_soup = bs4.BeautifulSoup(first_response.text, 'lxml')

<p style="font-family: Arial; font-size:1.5em;color:purple; font-style:bold">
分析具体文章页面的文档结构
</p>

<p style="font-family: Arial; font-size:1.5em;color:purple; font-style:bold">
标题
</p>
- html
  - body
    - div
      - div
        - div class="sub_conbox"
          - div class="con_nrtexttitle"
            - h2（标题）
          - div class="con_nrbox"
            - p
              - span（内容）

<p style="font-family: Arial; font-size:1.5em;color:purple; font-style:bold">
尝试获取标题
</p>

In [ ]:
first_title = first_soup.find('div', class_='con_nrtexttitle').find('h2')
first_title_str = str(first_title.string).strip()
first_title_str

<p style="font-family: Arial; font-size:1.5em;color:purple; font-style:bold">
从span标签获取内容
</p>

In [ ]:
# 定义一个函数，从span标签获取内容
def get_from_span(span_tag):
    result = ''
    for item in span_tag.contents:
        if isinstance(item, bs4.element.Tag):
            result += get_from_span(item)
        else:
            s = str(item).strip()
            if s:
                result += s
    return result

<p style="font-family: Arial; font-size:1.5em;color:purple; font-style:bold">
尝试获取内容
</p>

In [ ]:
# 获取内容部分的所有p标签
p_list = first_soup.find('div', class_='con_nrbox').find_all('p')

# 获取p标签下的span标签
span_list = []
for p in p_list:
    span_list.extend(p.find_all('span', recursive=False))

# 从span标签中提取内容
contents = []
for span in span_list:
    s = get_from_span(span)
    if s:
        contents.append(s)
' '.join(contents)

<p style="font-family: Arial; font-size:1.5em;color:purple; font-style:bold">
获取一篇文章定义为一个函数
</p>

将获取一篇文章的标题和内容的代码封装为一个函数，便于后面作为一个整体使用

In [ ]:
def get_article(url):
    response = requests.get(url)
    if response.status_code != 200:
        return ''
    response.encoding = 'utf-8'
    soup = bs4.BeautifulSoup(response.text, 'lxml')
    # 获取标题
    title = soup.find('div', class_='con_nrtexttitle').find('h2')
    title_str = str(title.string).strip()
    
    # 获取内容
    p_list = soup.find('div', class_='con_nrbox').find_all('p')
    span_list = []
    for p in p_list:
        span_list.extend(p.find_all('span', recursive=False))
    contents = []
    for span in span_list:
        s = get_from_span(span)
        if s:
            contents.append(s)
    content_str = ' '.join(contents)
    
    return title_str + ' ' + content_str

<p style="font-family: Arial; font-size:1.5em;color:purple; font-style:bold">
测试一下get_article函数
</p>

In [ ]:
get_article(first_url)

<p style="font-family: Arial; font-size:1.5em;color:purple; font-style:bold">
处理一个文章列表页面，定义为函数
</p>

将获取一个列表页的所有文章的代码封装为函数，便于后面作为整体使用。

In [ ]:
def get_soup(response):
    if response.status_code != 200:
        return None
    soup = bs4.BeautifulSoup(response.text, 'lxml')
    soup.encoding = 'utf-8'
    return soup

def get_article_list(soup):
    article_list = []
    ul_tag = soup.find("div", class_="sub_rbox").find("ul")
    a_list = ul_tag.find_all('a')
    for a in a_list:
        article_url = urljoin(response.url, a['href'])
        print(article_url)    # 用于调试
        article = get_article(article_url)
        article_list.append(article)
    return article_list

<p style="font-family: Arial; font-size:1.5em;color:purple; font-style:bold">
测试一下get_article_list函数
</p>

获取学校要闻第一页列表的所有文章标题和内容。

In [ ]:
response = requests.get('http://www.hbun.net/smartcore/web/main/xnxw/index.htm')
soup = get_soup(response)

In [ ]:
first_page = get_article_list(soup)
len(first_page)

<p style="font-family: Arial; font-size:1.5em;color:purple; font-style:bold">
获取下一列表页的地址
</p>

In [ ]:
def get_next_page_url(response, soup):
    a = soup.find('a', string='下一页')
    return urljoin(response.url, a['href'])

In [ ]:
get_next_page_url(response, soup)

<p style="font-family: Arial; font-size:1.5em;color:purple; font-style:bold">
获取前n页的文章信息
</p>

In [ ]:
def get_articles_n(start_url, n):
    articles = []
    list_url = start_url
    for i in range(n):
        print('******************* 第 %d 页 *******************' % i+1)
        response = requests.get(list_url)
        soup = get_soup(response)
        articles.extend(get_article_list(soup))
        list_url = get_next_page_url(response, soup)
    return articles

In [ ]:
article_list = get_articles_n('http://www.hbun.net/smartcore/web/main/xnxw/index.htm', 5)

In [ ]:
len(article_list)

<p style="font-family: Arial; font-size:1.5em;color:purple; font-style:bold">
将所有文章拼接起来，并分割为词条
</p>

In [ ]:
import jieba

In [ ]:
text = ' '.join(article_list)
 
word_list = []
for word in jieba.cut(text):
    word_list.append(word) 
len(word_list)

<p style="font-family: Arial; font-size:1.5em;color:purple; font-style:bold">
绘制词云
</p>

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [ ]:
words = ' '.join(word_list)
wc = WordCloud(font_path='yahei.ttf').generate(words)
plt.imshow(wc)
plt.axis("off")
plt.show()